***Gradients: The parameter-shift rule
100 points***

There are many ways of differentiating quantum circuits. One such way that is also hardware-compatible (i.e., it can be computed using a real quantum computer) is the parameter-shift rule. Here, we'll explore using the parameter-shift rule in the simplest case: for differentiating the Pauli rotation gates.

![title](gradients_formula1.png)

***Challenge code***

In the provided code, you are given a couple of functions:

![title](gradients_formula2.png)

***Input***

As input to this problem, you are given:

+ params (list(float)): a list corresponding to $\overrightarrow{\theta}$
* shift (float): the shift amount, corresponding to s.

***Output***

This code will output the gradient (list(float)) of the circuit.

If your solution matches the correct one within the given tolerance specified in check (in this case it's a 1e-4 relative error tolerance), the output will be "Correct!". Otherwise, you will receive a "Wrong answer" prompt.

Good luck!


***Code***

In [8]:
import functools
import json
import math
import pandas as pd
import pennylane as qml
import pennylane.numpy as np
import scipy

In [9]:
dev = qml.device("default.qubit", wires=2)

@qml.qnode(dev)
def circuit(params):
    """The quantum circuit that you will differentiate!

    Args:
        params (list(float)): The parameters for gates in the circuit

    Returns:
        (numpy.array): An expectation value.
    """
    qml.RY(params[0], 0)
    qml.RX(params[1], 1)
    return qml.expval(qml.PauliZ(0) + qml.PauliZ(1))

In [10]:
def my_parameter_shift_grad(params, shift):
    """Your homemade parameter-shift rule function.
    
    NOTE: you cannot use qml.grad within this function

    Args:
        params (list(float)): The parameters for gates in the circuit

    Returns:
        gradient (numpy.array): The gradient of the circuit with respect to the given parameters.
    """
    gradient = np.zeros_like(params)

    for i in range(len(params)):
        # Put your code here #
        for i in range(len(params)):
            shifted_params1 = np.copy(params)
            shifted_params1[i] += shift
            shifted_params2 = np.copy(params)
            shifted_params2[i] -= shift
            gradient[i] = (circuit(shifted_params1)-circuit(shifted_params2))/(2*math.sin(shift))
    return np.round_(gradient, decimals=5).tolist()

In [11]:
# These functions are responsible for testing the solution.

def run(test_case_input: str) -> str:
    params, shift = json.loads(test_case_input)
    gradient = my_parameter_shift_grad(params, shift)
    return str(gradient)

def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(
        solution_output, expected_output, rtol=1e-4
    ), "Your gradient isn't quite right!"

In [12]:
# These functions are responsible for testing the solution.

def run(test_case_input: str) -> str:
    params, shift = json.loads(test_case_input)
    gradient = my_parameter_shift_grad(params, shift)
    return str(gradient)

def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(
        solution_output, expected_output, rtol=1e-4
    ), "Your gradient isn't quite right!"

In [13]:
test_cases = [['[[0.75, 1.0], 1.23]', '[-0.68164, -0.84147]']]

In [14]:
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[[0.75, 1.0], 1.23]'...
Correct!
